<a href="https://colab.research.google.com/github/charlaburnett/dc_heat_analysis/blob/main/dc_heat_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import numpy as np
import rasterio

def calculate_lst_dual(tif_path):
    with rasterio.open(tif_path) as src:
        lst_dn = src.read(1).astype("float32")
        profile = src.profile

        lst_dn = np.where(lst_dn > 0, lst_dn, np.nan)


        lst_kelvin = (lst_dn * 0.00341802) + 149.0
        lst_celsius = lst_kelvin - 273.15
        lst_fahrenheit = (lst_celsius * 9 / 5) + 32

    return lst_celsius, lst_fahrenheit, profile


# 2015
lst_c_2015, lst_f_2015, prof_2015 = calculate_lst_dual(
    "/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/LC08_L2SP_015033_20150614_20201015_02_T1_ST_B10.TIF"
)

# 2025
lst_c_2025, lst_f_2025, prof_2025 = calculate_lst_dual(
    "/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/LC08_L2SP_015033_20250625_20250701_02_T1_ST_B10.TIF"
)



In [ ]:
#def save_lst(output_path, data, profile):
    #profile.update(dtype="float32", count=1)
    #with rasterio.open(output_path, "w", **profile) as dst:
        #dst.write(data, 1)

#save_lst("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/", lst_c_2015, prof_2015)
#save_lst("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/", lst_f_2015, prof_2015)
#save_lst("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/", lst_c_2025, prof_2025)
#save_lst("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/", lst_f_2025, prof_2025)


In [ ]:
import rasterio
from rasterio.features import shapes
import geopandas as gpd

dc_clipped = "/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/nlcd_2024_dc_clipped.tif"

with rasterio.open(dc_clipped) as src:
    band = src.read(1)
    mask = band != src.nodata

    results = (
        {"properties": {"nlcd_code": int(v)}, "geometry": s}
        for s, v in shapes(band, mask=mask, transform=src.transform)
    )

    gdf = gpd.GeoDataFrame.from_features(list(results), crs=src.crs)

def classify_uhi(code):
    if code in [21, 22, 23, 24]:
        return "Urban"
    elif code in [41, 42, 43, 52]:
        return "Vegetation"
    elif code == 11:
        return "Water"
    else:
        return "Other"

gdf["uhi_class"] = gdf["nlcd_code"].apply(classify_uhi)

output_path = "/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_landcover_polygons.shp"
gdf.to_file(output_path)

print(f"Saved classified land cover shapefile to:\n{output_path}")


In [ ]:
from rasterstats import zonal_stats
import geopandas as gpd
import rasterio

# Load your land cover classification polygons
landcover_gdf = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_landcover_polygons.shp")

# Reproject to match the LST raster's CRS
with rasterio.open("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/lst_2015_celsius.tif") as src_2015:
    landcover_2015 = landcover_gdf.to_crs(src_2015.crs)

# Zonal statistics
stats_2015 = zonal_stats(
    vectors=landcover_2015,
    raster="/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/lst_2015_celsius.tif",
    stats=["mean"],
    geojson_out=True
)

# Convert to GeoDataFrame
gdf_2015 = gpd.GeoDataFrame.from_features(stats_2015)
gdf_2015.set_crs(src_2015.crs, inplace=True)

# Save output
gdf_2015.to_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_stats_2015.shp")
print("✅ Saved 2015 zonal stats shapefile.")


In [ ]:
# Reproject polygons to match 2025 raster CRS
with rasterio.open("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/lst_2015_celsius.tif") as src_2025:
    landcover_2025 = landcover_gdf.to_crs(src_2025.crs)

# Zonal stats for 2025
stats_2025 = zonal_stats(
    vectors=landcover_2025,
    raster="/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/lst_2025_celsius.tif",
    stats=["mean"],
    geojson_out=True
)

# Convert to GeoDataFrame
gdf_2025 = gpd.GeoDataFrame.from_features(stats_2025)
gdf_2025.set_crs(src_2025.crs, inplace=True)
gdf_2025.to_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_stats_2025.shp")
print("✅ Saved 2025 zonal stats shapefile.")


In [ ]:
# Load both years' stats
gdf_2015 = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_stats_2015.shp")
gdf_2025 = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_stats_2025.shp")

# Ensure both are in WGS84
gdf_2015 = gdf_2015.to_crs(epsg=4326)
gdf_2025 = gdf_2025.to_crs(epsg=4326)

# Merge and compute temperature change
comparison = gdf_2015[["geometry", "mean"]].copy()
comparison = comparison.rename(columns={"mean": "mean_2015"})
comparison["mean_2025"] = gdf_2025["mean"]
comparison["temp_change"] = comparison["mean_2025"] - comparison["mean_2015"]

# Save comparison shapefile
comparison_path = "/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_change_2015_2025.shp"
comparison.to_file(comparison_path)
print(f"✅ Saved temp change shapefile to:\n{comparison_path}")


In [ ]:
import leafmap
import geopandas as gpd

# Load comparison shapefile
comparison = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_change_2015_2025.shp")
comparison = comparison.to_crs(epsg=4326)
comparison = comparison.dropna(subset=["mean_2015", "mean_2025", "temp_chang"])

# Initialize map
m = leafmap.Map(center=[38.9, -77.0], zoom=12)

# Add 2015 surface temperature
m.add_data(
    comparison,
    column="mean_2015",
    cmap="YlOrRd",
    layer_name="LST 2015 (°C)",
    legend_title="2015 Mean Temp (°C)",
    style={"weight": 0.2, "fillOpacity": 0.6}
)

# Add 2025 surface temperature
m.add_data(
    comparison,
    column="mean_2025",
    cmap="YlOrRd",
    layer_name="LST 2025 (°C)",
    legend_title="2025 Mean Temp (°C)",
    style={"weight": 0.2, "fillOpacity": 0.6}
)

# Add temperature change
m.add_data(
    comparison,
    column="temp_chang",
    cmap="coolwarm",
    layer_name="Temp Change (°C)",
    legend_title="Temp Change (2025 - 2015 °C)",
    style={"weight": 0.2, "fillOpacity": 0.7}
)

# Add layer control
m.add_layer_control()

# Show the map
m


In [ ]:
# Save to HTML
m.to_html("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_map.html")
print("✅ HTML map saved successfully!")


In [ ]:
from rasterstats import zonal_stats
import geopandas as gpd
import rasterio

landcover_gdf = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_landcover_polygons.shp")

# --- 2015 ---
with rasterio.open("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/lst_2015_fahrenheit.tif") as src_2015:
    landcover_2015 = landcover_gdf.to_crs(src_2015.crs)

stats_2015_f = zonal_stats(
    vectors=landcover_2015,
    raster="/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/lst_2015_fahrenheit.tif",
    stats=["mean"],
    geojson_out=True
)

gdf_2015_f = gpd.GeoDataFrame.from_features(stats_2015_f)
gdf_2015_f.set_crs(src_2015.crs, inplace=True)
gdf_2015_f.to_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_stats_2015_f.shp")

# --- 2025 ---
with rasterio.open("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat//lst_2025_fahrenheit.tif") as src_2025:
    landcover_2025 = landcover_gdf.to_crs(src_2025.crs)

stats_2025_f = zonal_stats(
    vectors=landcover_2025,
    raster="/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/lst_2025_fahrenheit.tif",
    stats=["mean"],
    geojson_out=True
)

gdf_2025_f = gpd.GeoDataFrame.from_features(stats_2025_f)
gdf_2025_f.set_crs(src_2025.crs, inplace=True)
gdf_2025_f.to_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_stats_2025_f.shp")


In [ ]:
gdf_2015_f = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_stats_2015_f.shp")
gdf_2025_f = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_stats_2025_f.shp")

# Continue with merge → map


In [ ]:
import geopandas as gpd

# Load zonal stats shapefiles (already created using Fahrenheit rasters)
gdf_2015_f = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_stats_2015_f.shp")
gdf_2025_f = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_stats_2025_f.shp")

# Set CRS if not present
if gdf_2015_f.crs is None:
    gdf_2015_f.set_crs(epsg=32618, inplace=True)
if gdf_2025_f.crs is None:
    gdf_2025_f.set_crs(epsg=32618, inplace=True)

# Reproject to WGS84
gdf_2015_f = gdf_2015_f.to_crs(epsg=4326)
gdf_2025_f = gdf_2025_f.to_crs(epsg=4326)

# Merge and calculate temperature change
comparison_f = gdf_2015_f[["geometry", "mean"]].copy()
comparison_f = comparison_f.rename(columns={"mean": "mean_2015_f"})
comparison_f["mean_2025_f"] = gdf_2025_f["mean"]
comparison_f["temp_change_f"] = comparison_f["mean_2025_f"] - comparison_f["mean_2015_f"]

# Save updated shapefile
comparison_f.to_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_change_2015_2025_fahrenheit.shp")


In [ ]:
import geopandas as gpd

comparison = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_change_2015_2025_fahrenheit.shp")
print(comparison.columns)


In [ ]:
import geopandas as gpd
import leafmap

# Load and prepare data
comparison = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/dc-urban-heat/uhi_temp_change_2015_2025_fahrenheit.shp")
comparison = comparison.to_crs(epsg=4326)
comparison_clean = comparison.dropna(subset=["mean_2015_", "mean_2025_", "temp_chang"])

# Initialize map
m = leafmap.Map(center=[38.9, -77.0], zoom=12)

# Add 2015 layer (°F)
m.add_data(
    comparison_clean,
    column="mean_2015_",
    cmap="YlOrRd",
    layer_name="LST 2015 (°F)",
    legend_title="2015 Mean Surface Temp (°F)",
    style={"weight": 0.2, "fillOpacity": 0.6}
)

# Add 2025 layer (°F)
m.add_data(
    comparison_clean,
    column="mean_2025_",
    cmap="YlOrRd",
    layer_name="LST 2025 (°F)",
    legend_title="2025 Mean Surface Temp (°F)",
    style={"weight": 0.2, "fillOpacity": 0.6}
)

# Add Temp Change layer (°F)
m.add_data(
    comparison_clean,
    column="temp_chang",
    cmap="coolwarm",
    layer_name="Temp Change (°F)",
    legend_title="Temp Change (°F)",
    style={"weight": 0.2, "fillOpacity": 0.7}
)

m.add_data(
    comparison_clean,
    column="temp_chang",
    cmap="coolwarm",
    layer_name="Temp Change (°F)",
    legend_title="Temp Change (°F)",
    style={"weight": 0.2, "fillOpacity": 0.7},
    legend_kw={
        "classes": 6,
        "colorbar": True
    }
)


m.add_layer_control()
m
